# Core Authentication Utilities

> Simple, transparent authentication utilities for FastHTML applications

In [ ]:
#| default_exp auth

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import bcrypt
import secrets
from typing import Optional, Dict, Any
from functools import wraps

## Password Hashing

Launch Kit provides simple utilities for secure password hashing using bcrypt. These functions are transparent and composable - you can see exactly what they do and use them however you need.

In [ ]:
#| export
def hash_password(password: str) -> str:
    """Hash a password using bcrypt with a cost factor of 12.
    
    Args:
        password: Plain text password to hash
        
    Returns:
        Hashed password as a string for database storage
    """
    # Cost factor of 12 is recommended for 2024
    # This takes ~250ms on modern hardware
    salt = bcrypt.gensalt(rounds=12)
    hashed = bcrypt.hashpw(password.encode('utf-8'), salt)
    return hashed.decode('utf-8')

In [ ]:
#| export
def verify_password(password: str, hashed: str) -> bool:
    """Verify a password against a bcrypt hash.
    
    Uses timing-safe comparison to prevent timing attacks.
    
    Args:
        password: Plain text password to verify
        hashed: Bcrypt hash from the database
        
    Returns:
        True if password matches, False otherwise
    """
    try:
        return bcrypt.checkpw(
            password.encode('utf-8'), 
            hashed.encode('utf-8')
        )
    except (ValueError, TypeError):
        # Invalid hash format
        return False

### Example Usage

Here's how you use these password utilities in your FastHTML application:

In [ ]:
# Hash a password when user signs up
password = "secure_password123"
hashed = hash_password(password)
print(f"Hashed password: {hashed}")

# Verify password when user logs in
is_valid = verify_password(password, hashed)
print(f"Password valid: {is_valid}")

# Test with wrong password
wrong_password = "wrong_password"
is_valid = verify_password(wrong_password, hashed)
print(f"Wrong password valid: {is_valid}")

## Session Management

These utilities work with FastHTML's session system to manage user authentication state.

In [ ]:
#| export
def user_auth_before(req, sess):
    """Beforeware function to check authentication status.
    
    Checks if user is authenticated by looking for 'auth' key in session.
    If authenticated, loads user data into the session.
    
    Args:
        req: FastHTML request object
        sess: FastHTML session object
        
    Usage:
        beforeware = Beforeware(
            user_auth_before,
            skip=['/auth/login', '/auth/signup', '/static/.*']
        )
    """
    # Check if user is authenticated
    if 'auth' in sess and sess.get('auth'):
        # User is authenticated, ensure user data is in session
        if 'user' not in sess:
            # In a real app, you'd fetch from database here
            # For now, we'll just ensure the structure exists
            user_id = sess.get('user_id')
            if user_id:
                # This is where you'd load user from database
                # sess['user'] = get_user_by_id(user_id)
                pass
    else:
        # Clear any stale user data if not authenticated
        sess.pop('user', None)
        sess.pop('user_id', None)

In [ ]:
#| export
def get_user_from_session(sess) -> Optional[Dict[str, Any]]:
    """Extract user data from session.
    
    Args:
        sess: FastHTML session object
        
    Returns:
        User dictionary if authenticated, None otherwise
    """
    if sess.get('auth') and 'user' in sess:
        return sess['user']
    return None

## Authentication Tokens

For remember-me functionality and API authentication, we provide simple token utilities.

In [ ]:
#| export
def create_auth_token(user_id: int) -> str:
    """Create a secure authentication token for a user.
    
    Args:
        user_id: ID of the user to create token for
        
    Returns:
        Secure random token string
        
    Note:
        In production, you'd store this token in your database
        with an expiration date and associate it with the user.
    """
    # Generate a secure random token
    token = secrets.token_urlsafe(32)
    # In a real app, you'd store: token -> user_id mapping in database
    return f"{user_id}:{token}"

In [ ]:
#| export
def verify_auth_token(token: str) -> Optional[int]:
    """Verify an authentication token and return the user ID.
    
    Args:
        token: Authentication token to verify
        
    Returns:
        User ID if token is valid, None otherwise
        
    Note:
        In production, you'd look up this token in your database
        and check expiration.
    """
    try:
        # Simple parsing for demo - in production, look up in database
        parts = token.split(':', 1)
        if len(parts) == 2:
            user_id = int(parts[0])
            # In real app: verify token exists in DB and not expired
            return user_id
    except (ValueError, TypeError):
        pass
    return None

### Token Example

In [ ]:
# Create a token for user ID 123
token = create_auth_token(123)
print(f"Auth token: {token}")

# Verify the token
user_id = verify_auth_token(token)
print(f"Verified user ID: {user_id}")

# Test with invalid token
invalid_token = "invalid_token"
user_id = verify_auth_token(invalid_token)
print(f"Invalid token result: {user_id}")

## Security Considerations

When using these authentication utilities:

1. **Password Requirements**: Implement minimum password requirements in your application
2. **Rate Limiting**: Use the rate limiting middleware to prevent brute force attacks
3. **Session Security**: Configure secure session cookies in production
4. **Token Storage**: Store auth tokens in a database with expiration times
5. **HTTPS Only**: Always use HTTPS in production to protect credentials in transit

## Integration with FastHTML

Here's a complete example of how to use these utilities in a FastHTML app:

```python
from fasthtml.common import *
from launch_kit.auth import hash_password, verify_password, user_auth_before

app, rt = fast_app()

# Set up authentication beforeware
beforeware = Beforeware(
    user_auth_before,
    skip=['/auth/login', '/auth/signup', '/static/.*']
)

@rt('/auth/signup')
def post(email: str, password: str, sess):
    # Hash the password
    hashed = hash_password(password)
    
    # Save user to database (pseudo-code)
    # user = create_user(email=email, password=hashed)
    
    # Set session
    sess['auth'] = True
    sess['user_id'] = user.id
    sess['user'] = {'id': user.id, 'email': email}
    
    return RedirectResponse('/')

@rt('/auth/login')
def post(email: str, password: str, sess):
    # Get user from database (pseudo-code)
    # user = get_user_by_email(email)
    
    if user and verify_password(password, user.password):
        # Login successful
        sess['auth'] = True
        sess['user_id'] = user.id
        sess['user'] = {'id': user.id, 'email': email}
        return RedirectResponse('/')
    else:
        # Login failed
        return "Invalid credentials", 401
```

These utilities are designed to be simple and transparent. You can see exactly what they do and modify them to fit your needs.

## Testing

In [ ]:
#| hide
# Test password hashing
test_password = "test_password_123!"
hashed = hash_password(test_password)

# Ensure hash is different from password
assert hashed != test_password
assert len(hashed) == 60  # bcrypt hashes are always 60 chars
assert hashed.startswith('$2b$')  # bcrypt prefix

# Test password verification
assert verify_password(test_password, hashed) == True
assert verify_password("wrong_password", hashed) == False

# Test with empty password
empty_hash = hash_password("")
assert verify_password("", empty_hash) == True
assert verify_password("not_empty", empty_hash) == False

# Test invalid hash handling
assert verify_password("any_password", "invalid_hash") == False
assert verify_password("any_password", "") == False

print("✓ Password hashing tests passed")

In [ ]:
#| hide
# Test token creation and verification
user_id = 42
token = create_auth_token(user_id)

# Token should contain user ID and random part
assert ':' in token
assert token.startswith(f"{user_id}:")

# Verify token returns correct user ID
verified_id = verify_auth_token(token)
assert verified_id == user_id

# Test invalid tokens
assert verify_auth_token("invalid") is None
assert verify_auth_token("") is None
assert verify_auth_token("no_colon") is None
assert verify_auth_token("not_a_number:token") is None

print("✓ Token tests passed")

In [ ]:
#| hide
# Test session utilities
mock_sess = {}

# Test get_user_from_session with no auth
assert get_user_from_session(mock_sess) is None

# Test with auth but no user
mock_sess['auth'] = True
assert get_user_from_session(mock_sess) is None

# Test with auth and user
mock_sess['user'] = {'id': 1, 'email': 'test@example.com'}
user = get_user_from_session(mock_sess)
assert user is not None
assert user['id'] == 1
assert user['email'] == 'test@example.com'

# Test user_auth_before
mock_req = None  # Not used in our implementation
test_sess = {'auth': True, 'user_id': 123}
user_auth_before(mock_req, test_sess)
# Should preserve auth and user_id
assert test_sess.get('auth') == True
assert test_sess.get('user_id') == 123

# Test cleanup when not authenticated
test_sess = {'user': 'stale_data', 'user_id': 999}
user_auth_before(mock_req, test_sess)
# Should clean up stale data
assert 'user' not in test_sess
assert 'user_id' not in test_sess

print("✓ Session management tests passed")

In [ ]:
#| hide
# Performance benchmark for password hashing
import time

start = time.time()
hash_password("benchmark_password")
duration = time.time() - start

print(f"Password hashing took: {duration:.3f} seconds")
print(f"This is {'acceptable' if duration < 0.5 else 'slow'} for production use")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()